In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import tensorflow as tf
config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.3
tf.Session(config=config)
# import seaborn as sns
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

%matplotlib inline

Using TensorFlow backend.


In [ ]:
train = pd.read_json('data/train.json')
test = pd.read_json('data/test.json')

In [3]:
train.shape, test.shape

((1604, 5), (8424, 4))

In [4]:
def get_scaled_imgs(df):
    imgs = []
    labels = []
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

        imgs.append(np.dstack((a, b, c)))
        labels.append(row['is_iceberg'])

    return np.array(imgs), np.array(labels)

In [5]:
def get_training_data(df, angles=False):
    imgs = []
    labels = []
    inc_angles = []
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 / band_2
#         band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
#         a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
#         b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
#         c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())
        
        a = (band_1 + abs(band_1.min())) / np.max((band_1 + abs(band_1.min())))
        b = (band_2 + abs(band_2.min())) / np.max((band_2 + abs(band_2.min())))
        c = (band_3 + abs(band_3.min())) / np.max((band_3 + abs(band_3.min())))
        
        imgs.append(np.dstack((a, b, c)))
        labels.append(row['is_iceberg'])
        if angles: 
            inc_angles.append(row['inc_angle'])
    if angles:    
        return np.array(imgs), np.array(inc_angles), np.array(labels)
    else:
        return np.array(imgs), np.array(labels)

In [6]:
Xtrain, Ytrain = get_scaled_imgs(train)

In [7]:
Xtest = get_test_imgs(test)

In [8]:
Xtrain.shape, Ytrain.shape, Xtest.shape

((1604, 75, 75, 3), (1604,), (8424, 75, 75, 3))

In [9]:
Ytrain = Ytrain.reshape(Ytrain.shape[0], 1)

In [10]:
def getModel():
    #Build keras model
    
    model=Sequential()
    
    # CNN 1
    model.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 2
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 3
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    #CNN 4
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # You must flatten the data for the dense layers
    model.add(Flatten())

    #Dense 1
    model.add(Dense(512, activation='relu'))
#     model.add(BatchNormalization())
    model.add(Dropout(0.2))

    #Dense 2
    model.add(Dense(256, activation='relu'))
#     model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
#     model.add(Dense(256, activation='relu'))
# #     model.add(BatchNormalization())
#     model.add(Dropout(0.25))
    # Output 
    model.add(Dense(1, activation="sigmoid"))

#     optimizer = Adam(lr=0.001, decay=0.0)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [11]:
# model = getModel()

In [12]:
# model.summary()
model = load_model('models/m1.h5')

In [13]:
%%time
model.fit(Xtrain, Ytrain, batch_size=4, epochs=20, verbose=1, validation_split=0.1)

Train on 1443 samples, validate on 161 samples
Epoch 1/20
1443/1443 [==============================] - 10s 7ms/step - loss: 0.1272 - acc: 0.9494 - val_loss: 0.3378 - val_acc: 0.8571
Epoch 2/20
1443/1443 [==============================] - 6s 4ms/step - loss: 0.1873 - acc: 0.9349 - val_loss: 0.2906 - val_acc: 0.8447
Epoch 3/20
1443/1443 [==============================] - 6s 4ms/step - loss: 0.1306 - acc: 0.9494 - val_loss: 0.3726 - val_acc: 0.8696
Epoch 4/20
1443/1443 [==============================] - 6s 4ms/step - loss: 0.1303 - acc: 0.9515 - val_loss: 0.2891 - val_acc: 0.8634
Epoch 5/20
1443/1443 [==============================] - 6s 4ms/step - loss: 0.1106 - acc: 0.9591 - val_loss: 0.2920 - val_acc: 0.8820
Epoch 6/20
1443/1443 [==============================] - 6s 4ms/step - loss: 0.1032 - acc: 0.9619 - val_loss: 0.5358 - val_acc: 0.8199
Epoch 7/20
1443/1443 [==============================] - 6s 4ms/step - loss: 0.1141 - acc: 0.9529 - val_loss: 0.3638 - val_acc: 0.8634
Epoch 8/20
144

In [58]:
acc = model.evaluate(Xtrain, Ytrain, verbose=1, batch_size=3)
print('Train score', acc[0])
print('Train accuracy', acc[1])

1604/1604 [==============================] - 3s 2ms/step
Train score 0.192378053727
Train accuracy 0.922069829264


In [61]:
model.save('models/m1.h5')

In [62]:
del model

### With data augmentation

In [14]:
train2, dev2 = train_test_split(train, test_size=0.1,random_state=0, stratify=train['is_iceberg'])

In [15]:
Xtrain2, Ytrain2 = get_scaled_imgs(train2)
Xdev2, Ydev2 = get_scaled_imgs(dev2)

In [16]:
Xtrain2.shape, Ytrain2.shape, Xdev2.shape, Ydev2.shape

((1443, 75, 75, 3), (1443,), (161, 75, 75, 3), (161,))

In [17]:
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.,
                         height_shift_range = 0.,
                         channel_shift_range=0,
                         zoom_range = 0.5,
                         rotation_range = 10)


In [18]:
gen_flow = gen.flow(Xtrain2, Ytrain2, batch_size=4, seed=5)

In [19]:
model2 = getModel()

In [20]:
model2.fit_generator(
    gen_flow,
    steps_per_epoch=1000,
    epochs=10,
    shuffle=True,
    verbose=1,
    validation_data=(Xdev2, Ydev2))

Epoch 1/10
1000/1000 [==============================] - 28s 28ms/step - loss: 0.6871 - acc: 0.5297 - val_loss: 0.6916 - val_acc: 0.5280
Epoch 2/10
1000/1000 [==============================] - 18s 18ms/step - loss: 0.6919 - acc: 0.5299 - val_loss: 0.6916 - val_acc: 0.5280
Epoch 3/10
1000/1000 [==============================] - 18s 18ms/step - loss: 0.6916 - acc: 0.5334 - val_loss: 0.6916 - val_acc: 0.5280
Epoch 4/10
1000/1000 [==============================] - 18s 18ms/step - loss: 0.6919 - acc: 0.5295 - val_loss: 0.6917 - val_acc: 0.5280
Epoch 5/10
1000/1000 [==============================] - 18s 18ms/step - loss: 0.6916 - acc: 0.5313 - val_loss: 0.6916 - val_acc: 0.5280
Epoch 6/10
1000/1000 [==============================] - 18s 18ms/step - loss: 0.6935 - acc: 0.5305 - val_loss: 0.6916 - val_acc: 0.5280
Epoch 7/10
1000/1000 [==============================] - 17s 17ms/step - loss: 0.6915 - acc: 0.5314 - val_loss: 0.6916 - val_acc: 0.5280
Epoch 8/10
1000/1000 [==========================

In [34]:
acc = model2.evaluate(Xtrain, Ytrain, verbose=1, batch_size=3)
print('Train score', acc[0])
print('Train accuracy', acc[1])

1604/1604 [==============================] - 4s 2ms/step
Train score 0.691317848731
Train accuracy 0.530548639521


In [21]:
Ypredicted = model.predict(Xtest, batch_size=5)

In [54]:
model2.save('models/m2.h5')

In [71]:
del model2

In [33]:
submission = pd.DataFrame({'id': test["id"], 'is_iceberg': Ypredicted.reshape((Ypredicted.shape[0])).round(decimals=3)})
submission.to_csv('submission.csv', index=False)

In [39]:
model.save()

In [16]:
# with open('submission-cnn-keras.csv', 'w') as file:
#     writer = csv.writer(file)
#     writer.writerow(['id', 'is_iceberg'])
#     for i, row in test.iterrows():
#         writer.writerow([row['id'], np.round(Ypredicted[i][0])])

In [14]:
model = load_model('models/m1.h5')

OSError: Unable to open file (unable to open file: name = 'models/m1.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)